In [5]:
import os
import os.path as op
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
%matplotlib inline
import seaborn as sns
sns.set_theme(style="darkgrid", color_codes=True)
sns.set(font_scale=1.35, style="ticks") #set styling preferences
import statsmodels.api as sm
from scipy import stats
import math
from math import pi
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype
from scipy.spatial.distance import cdist
from scipy.cluster.vq import kmeans2,vq, whiten
import geopandas as gpd
import h5py
import boto.s3
import glob
import boto3
from zipfile import ZipFile
import shutil
!pip install pyspark
from pyspark.sql import SparkSession

                                              0.0/310.8 MB ? eta -:--:--
                                              0.0/310.8 MB ? eta -:--:--
                                            0.0/310.8 MB 393.8 kB/s eta 0:13:09
                                              0.2/310.8 MB 1.7 MB/s eta 0:03:01
                                              0.6/310.8 MB 3.5 MB/s eta 0:01:29
                                              1.1/310.8 MB 4.9 MB/s eta 0:01:04
                                              1.7/310.8 MB 6.3 MB/s eta 0:00:50
                                              2.4/310.8 MB 7.6 MB/s eta 0:00:41
                                              3.3/310.8 MB 8.7 MB/s eta 0:00:36
                                             4.3/310.8 MB 10.7 MB/s eta 0:00:29
                                             5.7/310.8 MB 12.5 MB/s eta 0:00:25
                                             7.3/310.8 MB 14.6 MB/s eta 0:00:21
     -                                       9.4/310.8

In [2]:
# Show all columns and rows
pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [3]:
# Showing the entire number in dataframe
pd.set_option('float_format', '{:f}'.format)

#### Reading all the csv files

In [6]:
def read_csv_files():  
    sf_tel_base = pd.read_csv('gs://beam-core-outputs/sfbay-telecommuting/sfbay-telecommuting-baseline-20230616/inexus/sfbay_baseline_default-1.0_2020__20230616.csv.gz', compression = 'gzip')    # Adding scenario info
    sf_tel_base['scenario'] = "TELECOM"
    sf_tel_base['scenario'] = sf_tel_base['scenario'].astype("category")
    sf_tel_base['year'] = 2018
    sf_tel_base['lever_position_tel'] = 1      

    sf_tel_1p36 = pd.read_csv('gs://beam-core-outputs/sfbay-telecommuting/sfbay-telecommuting-0p36-20230620/inexus/sfbay_baseline_default-1.0_2020__20230620.csv.gz', compression = 'gzip')
    # Adding scenario info
    sf_tel_1p36['scenario'] = "TELECOM"
    sf_tel_1p36['scenario'] = sf_tel_1p36['scenario'].astype("category")
    sf_tel_1p36['year'] = 2018
    sf_tel_1p36['lever_position_tel'] = 1.36      
    
    sf_tel_1p76 = pd.read_csv('gs://beam-core-outputs/sfbay-telecommuting/sfbay-telecommuting-0p76-20230621/inexus/sfbay_baseline_default-1.0_2020__20230622.csv.gz', compression = 'gzip')
    # Adding scenario info
    sf_tel_1p76['scenario'] = "TELECOM"
    sf_tel_1p76['scenario'] = sf_tel_1p76['scenario'].astype("category")
    sf_tel_1p76['year'] = 2018
    sf_tel_1p76['lever_position_tel'] = 1.76        

    sf_tel_2p56 = pd.read_csv('gs://beam-core-outputs/sfbay-telecommuting/sfbay-telecommuting-1p56-20230617/inexus/sfbay_baseline_default-1.0_2020__20230618.csv.gz', compression = 'gzip')
    # Adding scenario info
    sf_tel_2p56['scenario'] = "TELECOM"
    sf_tel_2p56['scenario'] = sf_tel_2p56['scenario'].astype("category")
    sf_tel_2p56['year'] = 2018
    sf_tel_2p56['lever_position_tel'] = 2.56
    
    sf_tel_4p32 = pd.read_csv('gs://beam-core-outputs/sfbay-telecommuting/sfbay-telecommuting-3p32-20230621/inexus/sfbay_baseline_default-1.0_2020__20230621.csv.gz', compression = 'gzip')
    # Adding scenario info
    sf_tel_4p32['scenario'] = "TELECOM"
    sf_tel_4p32['scenario'] = sf_tel_4p32['scenario'].astype("category")
    sf_tel_4p32['year'] = 2018
    sf_tel_4p32['lever_position_tel'] = 4.32
   
    sf_tel_6p24 = pd.read_csv('gs://beam-core-outputs/sfbay-telecommuting/sfbay-telecommuting-5p24-20230617/inexus/sfbay_baseline_default-1.0_2020__20230618.csv.gz', compression = 'gzip')
    # Adding scenario info
    sf_tel_6p24['scenario'] = "TELECOM"
    sf_tel_6p24['scenario'] = sf_tel_6p24['scenario'].astype("category")
    sf_tel_6p24['year'] = 2018
    sf_tel_6p24['lever_position_tel'] = 6.24
    
    sf_tel_9p6 = pd.read_csv('gs://beam-core-outputs/sfbay-telecommuting/sfbay-telecommuting-8p60-20230620/inexus/sfbay_baseline_default-1.0_2020__20230620.csv.gz', compression = 'gzip')
    # Adding scenario info
    sf_tel_9p6['scenario'] = "TELECOM"
    sf_tel_9p6['scenario'] = sf_tel_9p6['scenario'].astype("category")
    sf_tel_9p6['year'] = 2018
    sf_tel_9p6['lever_position_tel'] = 9.6
    
    return (sf_tel_base, sf_tel_1p36, sf_tel_1p76, sf_tel_2p56, sf_tel_4p32, sf_tel_6p24, sf_tel_9p6)

In [7]:
%%time 
sf_tel_base, sf_tel_1p36, sf_tel_1p76, sf_tel_2p56, sf_tel_4p32, sf_tel_6p24, sf_tel_9p6 = read_csv_files()

C:\Users\nrezaei\AppData\Local\Temp\2\ipykernel_13188\1954552891.py:2: DtypeWarning: Columns (100) have mixed types. Specify dtype option on import or set low_memory=False.
  sf_tel_base = pd.read_csv('gs://beam-core-outputs/sfbay-telecommuting/sfbay-telecommuting-baseline-20230616/inexus/sfbay_baseline_default-1.0_2020__20230616.csv.gz', compression = 'gzip')    # Adding scenario info
C:\Users\nrezaei\AppData\Local\Temp\2\ipykernel_13188\1954552891.py:15: DtypeWarning: Columns (100) have mixed types. Specify dtype option on import or set low_memory=False.
  sf_tel_1p76 = pd.read_csv('gs://beam-core-outputs/sfbay-telecommuting/sfbay-telecommuting-0p76-20230621/inexus/sfbay_baseline_default-1.0_2020__20230622.csv.gz', compression = 'gzip')
C:\Users\nrezaei\AppData\Local\Temp\2\ipykernel_13188\1954552891.py:22: DtypeWarning: Columns (77,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,153,162,167,168,169,170,172,173,174,175,177,178,235,236,237,238,247) have mix

CPU times: total: 19min 28s
Wall time: 21min 54s


#### Stacked file

In [9]:
frames = [sf_tel_base, sf_tel_1p36, sf_tel_1p76, sf_tel_2p56, sf_tel_4p32, sf_tel_6p24, sf_tel_9p6]
sf_stacked = pd.concat(frames)

In [12]:
# Drop unnecessary columns (if any)
sf_stacked = sf_stacked.drop(['number_of_participants','tour_type_count',
                              'outbound','Unnamed: 0'], axis=1)

In [14]:
# Sort the stacked file
sf_stacked = sf_stacked.sort_values(by=['IDMerged', 'tripIndex', 'lever_position_tel']).reset_index(drop=True)

In [16]:
sf_stacked['door_to_door_time_relative_to_baseline'] = sf_stacked['duration_door_to_door'] - (sf_stacked['duration_door_to_door'].where(sf_stacked['lever_position_tel'].eq(1))
                                                                                                  .groupby([sf_stacked['IDMerged'],sf_stacked['tripIndex']])
                                                                                                   .transform('first')
                                                                                                   .convert_dtypes())

In [17]:
# Add the mandatory category column
mandatory = ['work' , 'univ', 'school']
sf_stacked['mandatoryCat'] = np.where((sf_stacked.actEndType.isin(mandatory)) & (sf_stacked.actStartType.isin(mandatory)), 'from_M_to_M' , None)

In [18]:
sf_stacked['mandatoryCat'] = np.where((sf_stacked.actEndType == 'Home') & (sf_stacked.actStartType.isin(mandatory)), 'from_H_to_M' , sf_stacked['mandatoryCat'])

In [19]:
sf_stacked['mandatoryCat'] = np.where((sf_stacked.actEndType.isin(mandatory)) & (sf_stacked.actStartType == "Home"), 'from_M_to_H' , sf_stacked['mandatoryCat'])

In [20]:
non_mandatory = ['othmaint' , 'othdiscr', 'escort', 'eatout', 'social', 'shopping', 'atwork']
sf_stacked['mandatoryCat'] = np.where((sf_stacked.actEndType.isin(non_mandatory)) & (sf_stacked.actStartType.isin(non_mandatory)), 'from_N_to_N' , sf_stacked['mandatoryCat'])

In [36]:
sf_stacked['socialCarbonCost'] = sf_stacked['emission_marginal']*185 # 51 or 102

#### Writing the stacked file to Google Cloud

In [48]:
%%time
import gzip
from io import BytesIO
from google.cloud import storage
import pandas as pd

# Export DataFrame to a gzip-compressed CSV file
compressed_bytes = BytesIO()
with gzip.GzipFile(fileobj=compressed_bytes, mode='w') as gz:
    sf_stacked.to_csv(gz, index=False)
compressed_bytes.seek(0)

# Upload the gzip-compressed file to Google Cloud Storage
storage_client = storage.Client.from_service_account_json('C:/Users/nrezaei/Documents/beam-core-a9ea929e82b1.json')
bucket_name = 'beam-core-analysis'  # Replace with your Google Cloud Storage bucket name
destination_blob_name = 'sf_2018_stacked_telecommuting_20230625.csv.gz'  # Replace with the desired destination file name in the bucket

bucket = storage_client.bucket(bucket_name)
blob = bucket.blob(destination_blob_name)
blob.upload_from_file(compressed_bytes, content_type='application/gzip')

# Set the Content-Encoding metadata property
blob.metadata = {'Content-Encoding': 'gzip'}
blob.patch()

CPU times: total: 1h 5min 35s
Wall time: 1h 5min 51s
